In [32]:
import os
from itertools import chain

fpath_win = 'C:\\Users\\rwmil\\Documents\\GitHub\\llm\\llm\\'
fpath_mac = '/Users/richardmiller/Documents/GitHub/llm/'
os.chdir(fpath_mac)
print(os.getcwd())

/Users/richardmiller/Documents/GitHub/llm


In [ ]:
with open('./LLMs-from-scratch-main/ch02/01_main-chapter-code/the-verdict.txt','r',encoding='utf-8') as f:
    raw_text = f.read()
f.close()

split_text = [re.split(r'([,.:;?_!"()\']|--|\s)',i) for i in raw_text.split()]
split_text = [i.strip() for item in split_text for i in item if i not in ['',' ']]
split_text.extend(['<|endoftext|>','<|unk|>'])

split_text = sorted(list(set(split_text)))
vocab = {token:i for i,token in enumerate(split_text)}

In [ ]:
class SimpleTokenizer:
    def __init__(self,vocab):
        ''' The vocab object must be a dictionary or 
        dictionary-like object and must be defined 
        outside of the tokenizer class. It also must
        have the form of {string:integer}'''
        self.string_to_int = vocab
        self.int_to_string = {i:s for s,i in vocab.items()}
        self.re_string_encoder = r'([,.:;?_!"()\']|--|\s)' #Leave these here because I can use them later to define the regex
        self.re_string_decoder = r'\s+([,.:;?!"()\'])' # strings that I want to use later. If my texts become more complex.

    def encoder(self,text):
        preprocess = re.split(self.re_string_encoder,text)
        preprocess = [i.strip() for i in preprocess if i not in ['',' ']]
        preprocess = [i if i in self.string_to_int else '<|unk|>' for i in preprocess]
        
        
        return [self.string_to_int[s] for s in preprocess]

    def decoder(self,ids):
        text = ' '.join([self.int_to_string[i] for i in ids])
        text = re.sub(self.re_string_decoder,r'\1',text)
        return text

tokenizer = SimpleTokenizer(vocab)
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
te = " <|endoftext|> ".join((text1, text2))
out = tokenizer.encoder(te)
out2 = tokenizer.decoder(out)

In [ ]:
print(out2)

In [ ]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

In [ ]:
tokenizer = tiktoken.get_encoding('gpt2')
text = ('Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.')
integers = tokenizer.encode(text,allowed_special={'<|endoftext|>'})
print(integers)

In [ ]:
strings = tokenizer.decode(integers)
print(strings)

## Section 2 Using Byte Pair Encoding

In [ ]:
import os
import tiktoken

fpath_win = 'C:\\Users\\rwmil\\Documents\\GitHub\\llm\\llm\\'
fpath_mac = '/Users/richardmiller/Documents/GitHub/llm/'
os.chdir(fpath_mac)
print(os.getcwd())

In [ ]:
#Get text data
with open('./LLMs-from-scratch-main/ch02/01_main-chapter-code/the-verdict.txt') as f:
    raw_text = f.read()
    
f.close()

#Initialize tokenizer and encode raw text from file
tokenizer = tiktoken.get_encoding('gpt2')
encoded_text = tokenizer.encode(raw_text)

#Remove last 50 characters
encoded_sample = encoded_text[:50]


In [ ]:
#Split encoded sample into x and y targets

context_size = 4 #Determines number of tokens included in input
x = encoded_sample[:context_size]
y = encoded_sample[1:context_size+1]

for i in range(1,context_size+1):
    context = encoded_sample[:i]
    desired = encoded_sample[i]
    print(context,'----->',desired)
    

In [ ]:
for i in range(1,context_size+1):
    context = encoded_sample[:i]
    desired = encoded_sample[i]
    print(tokenizer.decode(context),'----->',tokenizer.decode([desired]))

## Section 3

In [33]:
import torch
from torch.utils.data import DataLoader, Dataset
import tiktoken


In [34]:

class GPTDatasetV1(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):
        '''
        Looks like tokenizer is instantiated before being passed to this class.
        '''
        
        self.input_ids = []
        self.target_ids = []
        
        #Get token id's
        self.token_ids = tokenizer.encode(txt)
        
        for i in range(0,len(self.token_ids)-max_length,stride):
            input_chunk = self.token_ids[i:i+max_length]
            target_chunk = self.token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self,idx):
        return self.input_ids, self.target_ids
    
    
        

In [35]:
def create_dataloader_v1(
    txt,
    batch_size=25,
    max_length=255,
    stride=128,
    shuffle=True,
    drop_last=True,
    num_workers=0,
):
    tokenizer = tiktoken.get_encoding('gpt2')
    dataset = GPTDatasetV1(txt,tokenizer,max_length,stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    
    return dataloader



In [36]:
with open('./LLMs-from-scratch-main/ch02/01_main-chapter-code/the-verdict.txt') as f:
    raw_data = f.read()
    
f.close()

#max_length and stride can be changed, using a larger stride will
#result in less overfitting

#batch_size is a tunable hyperparameter to make the model better.
#Higher batch_size means more memory usage and less noise
#Lower batch_size means less memory usage but more nosie in training

dataloader = create_dataloader_v1(
    raw_data,
    batch_size=1,
    max_length=4,
    stride=1,
    shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)


In [37]:
input_ids = torch.tensor([2,3,5,1])
vocab_size = 6
output_dim = 3
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size,output_dim)



In [74]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size,output_dim)
max_length = 4
dataloader = create_dataloader_v1(raw_data,batch_size=8,max_length=max_length,stride=max_length,shuffle=False)
data_iter = iter(dataloader)
inputs,targets = next(data_iter)

token_embeddings = token_embedding_layer(inputs[1])

context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length,output_dim)
pos_embedding = pos_embedding_layer(torch.arange(context_length))
print(pos_embedding.shape,token_embedding.shape)

torch.Size([4, 256]) torch.Size([8, 4, 256])


In [68]:
input_embeddings = token_embeddings+pos_embedding

# Chapter 3 - Self-Attention

In [82]:
import torch
inputs = torch.tensor([
    [0.43,0.15,0.89],
    [0.55,0.87,0.66],
    [0.57,0.85,0.64],
    [0.22,0.58,0.33],
    [0.77,0.25,0.10],
    [0.05,0.80,0.55],
])

query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i,query)
    
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [83]:
attn_weights_2_tmp = attn_scores_2/attn_scores_2.sum()
print('Attention Weights: ',attn_weights_2_tmp)
print('Sum: ',attn_weights_2_tmp.sum())

Attention Weights:  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum:  tensor(1.0000)


In [85]:
#Computing attention weights of a single token by scalar multiplication of a vector
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i
    
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


In [87]:
#Computing attention weights of all tokens by dot product
attn_scores = torch.empty(6,6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i,j] = torch.dot(x_i,x_j)
        
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [88]:
attn_scores = inputs@inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [89]:
attn_weights = torch.softmax(attn_scores,dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])
